In [32]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
%pip install -qU "langchain[google-genai]"
%pip install -qU langchain-huggingface
%pip install -qU langchain-core
%pip install --upgrade --quiet  langchain langchain-community azure-ai-documentintelligence
%pip install python-dotenv
%pip install gradio --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
import bs4
from langchain import hub
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from dotenv import load_dotenv
import gradio as gr

load_dotenv()


True

In [2]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
document_intelligence_key = os.getenv("DOCUMENTAI_API_KEY")
file_path = "test.pptx"
endpoint = os.getenv("DOCUMENTAI_ENDPOINT")

### Chat Model - Gemini GenAI

In [3]:
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

### Embeddings model - HuggingFace

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

### Memory Vector Model - LangChain Core

In [5]:
vector_store = InMemoryVectorStore(embeddings)

In [6]:
loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=endpoint, api_key=document_intelligence_key, file_path=file_path, api_model="prebuilt-layout"
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

_ = vector_store.add_documents(documents=all_splits)

prompt = hub.pull("rlm/rag-prompt")

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [7]:

def chat_fn(question: str) -> str:
    # call your langgraph pipeline
    result = graph.invoke({"question": question})
    return result["answer"]

In [8]:
iface = gr.Interface(
    fn=chat_fn,
    inputs=gr.Textbox(lines=2, placeholder="Type your question here…"),
    outputs=gr.Textbox(label="Answer"),
    title="📄 Document Q & A Chatbot",
    description="Ask me anything about your loaded PPTX!"
)

iface.launch(share=True, inline=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [30]:
response = graph.invoke({"question": "List 3 generally reusable solution to a recurring problem?"})
print(response["answer"])

Three generally reusable solutions to recurring problems are design patterns, templates, and best practices. Design patterns offer a standard approach. Templates provide a structured solution.
